## Import dependencies

In [1]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
#import lambda
pymysql.install_as_MySQLdb()

## Import configuration variables

In [2]:
# import sys
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

## Set up `simple_salesforce`

Pass in the username, password, and security token here. The security token contains information about your org.

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [5]:
course_data = pd.read_sql("SELECT * FROM course", conn)
course_data.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


## Transform Data

In [6]:
course_data = course_data.rename(columns={'ID_Course__c':'ID_Course',
                                          'CourseCode__c':'CourseCode',
                                         'CreditHours__c':'CreditHours'})
course_data.head()

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [7]:
course_load_df = course_data[['ID_Course','CourseCode','CreditHours']]
course_load_df.head()

,ID_Course,CourseCode,CreditHours
0,1,BC-DATAVIZ,12
1,2,BC-WEBDEV,12
2,3,BC-UIUX,12
3,4,CIS-349,5
4,5,CIS-405,5


## Convert DataFrame to a list of dictionaries 

This can help facilitate a bulk insert

In [8]:
course_load = course_load_df.to_dict('records')
course_load

[{'ID_Course': 1, 'CourseCode': 'BC-DATAVIZ', 'CreditHours': 12},
 {'ID_Course': 2, 'CourseCode': 'BC-WEBDEV', 'CreditHours': 12},
 {'ID_Course': 3, 'CourseCode': 'BC-UIUX', 'CreditHours': 12},
 {'ID_Course': 4, 'CourseCode': 'CIS-349', 'CreditHours': 5},
 {'ID_Course': 5, 'CourseCode': 'CIS-405', 'CreditHours': 5},
 {'ID_Course': 6, 'CourseCode': 'CIS-438', 'CreditHours': 5},
 {'ID_Course': 7, 'CourseCode': 'CIS-430', 'CreditHours': 5},
 {'ID_Course': 8, 'CourseCode': 'CIS-435', 'CreditHours': 5}]

## Example of inserting rows individually - Course Data

Loop through the DataFrame and assemble indiviudal dictionaries

In [9]:
#for index, row in course_data.iterrows():
    record = {
        'ID_Course__c': row['ID_Course'],
        'CourseCode__c': row['CourseCode'],
        'CreditHours__c':row['CreditHours'],
        'BootCampCourse__c':row['BootCampCourse']
    }
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

IndentationError: unexpected indent (<ipython-input-9-587cd6db7d34>, line 2)

## Example of inserting rows in bulk

In [ ]:
#try:
#    sf.bulk.Lead.insert(course_load)
#except Exception as e:
#    print(e)

## Now the staff data 

In [ ]:
staff_data = pd.read_sql("SELECT * FROM staff", conn)
staff_data.head(30)

## Convert the dates to push data

In [ ]:
staff_data['Birth_Date'] = staff_data['BirthDate'].apply(lambda x: x.strftime('%Y-%m-%d')) 
staff_data=staff_data[['ID_Staff','EmployeeID','LastName','FirstName','MiddleName','BirthDate']]
staff_data

In [ ]:
#for index, row in staff_data.iterrows():
    record = {
        'ID_Staff__c': row['ID_Staff'],
        'EmployeeID__c': row['EmployeeID'],
        'LastName__c':row['LastName'],
        'FirstName__c':row['FirstName'],
        'MiddleName__c':row['MiddleName'],
    }
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

## Now the class data 

In [ ]:
class_data = pd.read_sql("SELECT * FROM class", conn)
class_data.head(30)

In [ ]:
class_data['StartDate'] = class_data['StartDate'].apply(lambda x: x.strftime('%Y-%m-%d')) 
class_data['EndDate'] = class_data['EndDate'].apply(lambda x: x.strftime('%Y-%m-%d')) 
class_data=class_data[['ID_Class','ID_Course','Section','StartDate','EndDate']]
class_data

In [ ]:
#for index, row in class_data.iterrows():
    record = {
        'ID_Class__c': row['ID_Class'],
        'ID_Course__c': row['ID_Course'],
        'Section__c':row['Section'],
        'StartDate__c':row['StartDate'],
        'EndDate__c':row['EndDate']
    }
    try:
        sf.Class__c.create(record)
    except Exception as e:
        print(e)

## Now the class participant data 

In [10]:
participant_data = pd.read_sql("SELECT * FROM classparticipant", conn)
participant_data.head(10)

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None
5,6,37,1,2020-03-16,None
6,7,38,1,2020-03-16,None
7,8,39,1,2020-03-16,None
8,9,40,1,2020-03-16,None
9,10,41,1,2020-03-16,None


In [17]:
#participant_data['StartDate'] = participant_data['StartDate'].apply(lambda x: x.strftime('%Y-%m-%d')) 
#participant_data['EndDate'] = participant_data['EndDate'].apply(lambda x: x.strftime('%Y-%m-%d')) 
participant_data=participant_data[['ID_ClassParticipant','ID_Student','ID_Class','StartDate','EndDate']]
participant_data.head(10)

,ID_ClassParticipant,ID_Student,ID_Class,StartDate,EndDate
0,1,33,1,2020-03-16,None
1,2,34,1,2020-03-16,None
2,3,35,1,2020-03-16,None
3,4,62,1,2020-03-16,None
4,5,36,1,2020-03-16,None
5,6,37,1,2020-03-16,None
6,7,38,1,2020-03-16,None
7,8,39,1,2020-03-16,None
8,9,40,1,2020-03-16,None
9,10,41,1,2020-03-16,None


In [20]:
for index, row in participant_data.iterrows():
    record = {
        'ID_ClassParticipant__c': row['ID_ClassParticipant'],
        'ID_Student__c': row['ID_Student'],
        'ID_Class__c':row['ID_Class'],
        'StartDate__c':row['StartDate'],
        'EndDate__c':row['EndDate']
    }
    try:
        sf.Classparticiant__c.create(record)
    except Exception as e:
        print(e)

## Now the staff assignment data

In [21]:
assignment_data = pd.read_sql("SELECT * FROM staffassignment", conn)
assignment_data.head(10)

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [25]:
assignment_data['StartDate'] = assignment_data['StartDate'].apply(lambda x: x.strftime('%Y-%m-%d')) 
assignment_data=assignment_data[['ID_StaffAssignment','ID_Staff','ID_Class', 'Role','StartDate','EndDate']]
assignment_data.head(10)

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-03-16,None
1,2,4,1,Teacher Assistant,2020-03-16,None
2,3,3,1,Instructor,2020-03-16,None


In [26]:
for index, row in assignment_data.iterrows():
    record = {
        'ID_StaffAssignment__c': row['ID_StaffAssignment'],
        'ID_Staff__c': row['ID_Staff'],
        'ID_Class__c':row['ID_Class'],
        'Role__c':row['Role'],
        'StartDate__c':row['StartDate'],
        'EndDate__c':row['EndDate']
    }
    try:
        sf.Staffassignment__c.create(record)
    except Exception as e:
        print(e)

## Now the student data

In [27]:
student_data = pd.read_sql("SELECT * FROM student", conn)
student_data.head(10)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F
5,38,25006014,Poocharoen,Pariya,Mariette,None,F
6,39,25007528,Mantrala,Sriharitha,Patty,None,F
7,40,25003778,Kelly,Carly,Raelene,None,F
8,41,25003605,Sraha,Clementine,Kendall,None,F
9,42,25002056,Abdulrahim,Jawaher,Angela,None,F


In [29]:
student_data=student_data[['ID_Student','StudentID','LastName', 'FirstName','MiddleName','BirthDate','Gender']]
student_data.head(10)

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F
5,38,25006014,Poocharoen,Pariya,Mariette,None,F
6,39,25007528,Mantrala,Sriharitha,Patty,None,F
7,40,25003778,Kelly,Carly,Raelene,None,F
8,41,25003605,Sraha,Clementine,Kendall,None,F
9,42,25002056,Abdulrahim,Jawaher,Angela,None,F


In [31]:
for index, row in student_data.iterrows():
    record = {
        'ID_Student__c': row['ID_Student'],
        'StudentID__c': row['StudentID'],
        'LastName__c':row['LastName'],
        'Name':row['FirstName'],
        'MiddleName__c':row['MiddleName'],
        'BirthDate__c':row['BirthDate'],
        'Gender__c':row['Gender']
    }
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)